In [1]:
import re
import json
import pandas as pd

In [2]:
data = pd.read_csv("../../data/data_cleaned_sentences_phases_2020-04-16.csv", sep='|', converters={'sentences': pd.eval})

In [3]:
data["title"] = data["title"].apply(lambda title: title if isinstance(title, str) else "") 
data["text"] = data.apply(lambda row: " ".join([sent for sent in [row["title"]] + row["sentences"]]), axis=1)

In [236]:
sample_data = data.sample(200)
sample_data

,id,date,pub_name,type,title,content,Label,sentences,text,phase
16954,17002,04 Ocak 2017 Çarşamba,dünya,Ulusal,muhalifler astana müzakerelerini boykot etme k...,hassas ateşkesin sonu gelebilir muhalifler ast...,not_hate,[hassas ateşkesin sonu gelebilir muhalifler as...,muhalifler astana müzakerelerini boykot etme k...,train
23393,23443,17 Temmuz 2019 Çarşamba,ilkses gazetesi,yerel,cezaevindeki çocuklara bez bebek arkadaşlar,cezaevindeki çocuklara bez bebek arkadaşlar iz...,not_hate,[cezaevindeki çocuklara bez bebek arkadaşlar i...,cezaevindeki çocuklara bez bebek arkadaşlar ce...,train
22990,23040,30 Mayıs 2019 Perşembe,trakya demokrat gazetesi,yerel,şehadet kelim esi,yazılarım akif can islamın ilk güzergahı ve an...,not_hate,[yazılarım akif can islamın ilk güzergahı ve a...,şehadet kelim esi yazılarım akif can islamın i...,test
9423,9464,02 Temmuz 2019 Salı,denge samsun,yerel,bilek güreşi,dursen ozalemdar tefekkür bilek güreşi ülkemiz...,hate,[dursen ozalemdar tefekkür bilek güreşi ülkemi...,bilek güreşi dursen ozalemdar tefekkür bilek g...,train
15167,15215,01 Ağustos 2017 Salı,karar,Ulusal,yardımcı doçentlik ne demek,yardımcı doçentlik ne demek? doktora ile doçen...,not_hate,"[yardımcı doçentlik ne demek, doktora ile doçe...",yardımcı doçentlik ne demek yardımcı doçentlik...,val
...,...,...,...,...,...,...,...,...,...,...
4802,4829,19 Ekim 2017 Perşembe,diyarbakır söz,yerel,günün,günün yorumu mehmet ali altındağ müstakil ve b...,hate,[günün yorumu mehmet ali altındağ müstakil ve ...,günün günün yorumu mehmet ali altındağ müstaki...,train
16312,16360,20 Haziran 2017 Salı,denizli horoz,Yerel,ikinci yolcu anıl,ikinci yolcu anıl denizlispor'da sezon başında...,not_hate,[ikinci yolcu anıl denizlisporda sezon başında...,ikinci yolcu anıl ikinci yolcu anıl denizlispo...,train
21448,21496,15 Ağustos 2018 Çarşamba,mayıs gazetesi,Yerel,allah bu güzel kardeşlerimizin sıkıntılarını a...,allah bu güzel kardeşlerimizin sıkıntılarını a...,not_hate,[allah bu güzel kardeşlerimizin sıkıntılarını ...,allah bu güzel kardeşlerimizin sıkıntılarını a...,train
13760,13808,08 Nisan 2016 Cuma,karar,Ulusal,kupa bu kez gelecek,kupa fenerbahçe kadın basketbol takımı oyuncul...,not_hate,[kupa fenerbahçe kadın basketbol takımı oyuncu...,kupa bu kez gelecek kupa fenerbahçe kadın bask...,train


In [251]:
class LinguisticRuleAssigner:
    def __init__(self):
        self.general_rules = [        
            r"\s[IRK](?!istan|stan|ya)(lık|lik|luk|lük)\s(.*?)(yap)",
            r"\s[IRK_KIMLIK](?!istan|stan|ya)\s(.*?)(skandalı)",
            r"\s[KIMLIK](?!istan|stan|ya)(ların|lerin|in|ın|un|ün)?\s(.*?)(işgali)",
            r"\s[IRK_KIMLIK](?!istan|stan|ya)(a|e)\s(bak sen)",
            r"\s[IRK](?!istan|stan|ya)(ların|lerin|in|ın|un|ün)?\s(.*?)(işgali)",
            r"\s[IRK](?!istan|stan|ya)\s(.*?)\s[IRK](?!istan|stan|ya)(lığını|liğini|luğunu|lüğünü)\s(.*?)(yap)",
            r"\s[IRK](?!istan|stan|ya)(ın|in|un|ün)\s(.*?)(uşağı|işbirlikçisi|piyonu|kuklası)(.*?)\s[IRK](?!istan|stan|ya)\s",
            r"\s[IRK](?!istan|stan|ya)(ın|in|un|ün)\s(.*?)(hain|vahşi|insanlık dışı|hunharca|kan donduran|şeytani|sinsi|[ADJBEF])\s(teşebbüsleri|planları|oluşumları|[ADJAFTER])",
            r"\s[IRK](?!istan|stan|ya)\s(.*?)(kurşunlarıyla|bombalarıyla|parasıyla)",
            r"\s[IRK](?!istan|stan|ya)(ın|in|un|ün)\s(.*?)(gerçekleştirdiği|yaptığı)(.*?)(katliam|zulüm|soykırım|[ADJAFTER])",
            r"\s[IRK](?!istan|stan|ya)\s(tarafından)\s(gerçekleştirilen|yapılan)(.*?)(saldırılar|katliam|zulüm|soykırım|[ADJAFTER])",
            r"\s[IRK](?!istan|stan|ya)\s(tarafından)\s[IRK](?!istan|stan|ya)(a|e)(yönelik)?(.*?)(saldırılar|katliam|zulüm|soykırım|[ADJAFTER])",
            r"\s[IRK](?!istan|stan|ya)\s(tarafından)(.*?)(saldırıya|katliama|zulme|soykırıma)(.*?)(maruz kal|uğra)",
            r"\s[IRK]\s(destekli)(.*?)\s[IRK](?!istan|stan|ya)\s(darbesi|saldırıları|katliamı|soykırımı)",
            r"\s[IRK](?!istan|stan|ya)\s(.*?)(öldürdü|katletti|etnik temizlik yaptı|kirletti|bastı|şehit etti)",
            r"\s[IRK](?!istan|stan|ya)\s(tarafından)(.*?)(öldürüldü|katledildi|basıldı|şehit edildi)"
        ]
        self.anti_hs_rules = [
            r"""\s[IRK](?!istan|stan|ya)\s(çeteleri|fanatikler|fanatiği|askerleri|milisleri|militanları|yerleşimciler|milliyetçiler|güçleri|isyanı|yaygaracılığı|polisi)""",
            r"(radikal|ırkçı|fanatik)\s[IRK](?!istan|stan|ya)\s",
            r"\s[IRK](?!istan|stan|ya)(cı|ci|çi|cu|cü|çü)\sterör\sörgütü"
        ]
        
        self.special_patterns = json.load(open("special_patterns.json", "r"))
        self.hs_specific_verbs = json.load(open("hs_specific_verbs.json", "r"))
        self.anti_hs_patterns = json.load(open("anti_hs.json", "r"))
        self.adj_bef_keyword = json.load(open("adjacent_before_keyword.json", "r"))
        self.adj_after_keyword = json.load(open("adjacent_after_keyword.json", "r"))
        
        self.keywords = json.load(open("targeted_keywords.json", "r"))
        self.irk = "(" + "|".join(self.keywords["IRK"]) + ")"
        self.kimlik = "(" + "|".join(self.keywords["KIMLIK"]) + ")"
        self.irk_kimlik = "(" + "|".join(self.keywords["IRK"] + self.keywords["KIMLIK"]) + ")"
        
        self.adjbef = "|".join([item for sublist in list(self.adj_bef_keyword.values()) for item in sublist])
        self.adjafter = "|".join([item for sublist in list(self.adj_after_keyword.values()) for item in sublist])
        
        self.turkish_to_latin_map = {ord("ü"): "u", ord("ö"): "o", ord("ı"): "i", ord("ğ"): "g", ord("ş"): "s", ord("ç"): "c"}
        
        self.__prepare_rules()
    
    def __add_keyword_into_rule(self, rules):
        for i in range(len(rules)):
            if "[IRK_KIMLIK]" in rules[i]:
                rules[i] = rules[i].replace("[IRK_KIMLIK]", self.irk_kimlik)
            if "[IRK]" in rules[i]:
                rules[i] = rules[i].replace("[IRK]", self.irk)
            if "[KIMLIK]" in rules[i]:
                rules[i] = rules[i].replace("[KIMLIK]", self.kimlik)
            if "[ADJBEF]" in rules[i]:
                rules[i] = rules[i].replace("[ADJBEF]", self.adjbef)
            if "[ADJAFTER]" in rules[i]:
                rules[i] = rules[i].replace("[ADJAFTER]", self.adjafter)
        return rules 
    
    def __prepare_rules(self):
        self.general_rules = self.__add_keyword_into_rule(self.general_rules)
        self.anti_hs_rules = self.__add_keyword_into_rule(self.anti_hs_rules)
    
    def __detect_special_pattern(self, text):
        pattern_list = []
        for _, patterns in self.special_patterns.items():
            if len(patterns) != 0:
                patterns = f"\s({'|'.join(patterns)})|({'|'.join(patterns)})\s"
                patterns = r'{}'.format(patterns)
                found_texts = re.findall(patterns, text, flags=re.I)
                if len(found_texts) > 0:
                    pattern_list.append(1)
                else:
                    pattern_list.append(0)
            else:
                pattern_list.append(0)
        return pattern_list
    
    def __detect_general_rules(self, text):
        nof_rules_detected = 0
        for rule in self.general_rules:
            found_texts = re.findall(rule, text, flags=re.I)
            if len(found_texts) > 0:
                nof_rules_detected += 1
        return nof_rules_detected / len(self.general_rules)
    
    def __detect_anti_hs(self, text):
        pattern_list = []
        for degree, patterns in self.anti_hs_patterns.items():
            if len(patterns) != 0:
                found_rule = False
                if degree == "20":
                    for rule in self.anti_hs_rules:
                        found_texts = re.findall(rule, text, flags=re.I)
                        if len(found_texts) > 0:
                            found_rule = True
                            break
                patterns = f"\s({'|'.join(patterns)})|({'|'.join(patterns)})\s"
                patterns = r'{}'.format(patterns)
                found_texts = re.findall(patterns, text, flags=re.I)
                if len(found_texts) > 0 or found_rule:
                    pattern_list.append(1)
                else:
                    pattern_list.append(0)
            else:
                pattern_list.append(0)
        return pattern_list
        
    def __detect_bef_adj(self, text):
        pattern_list = []
        for _, patterns in self.adj_bef_keyword.items():
            if len(patterns) != 0:
                patterns = f"\s({'|'.join(patterns)})\s{self.irk}(?!istan|stan|ya)\s"
                patterns = r'{}'.format(patterns)
                found_texts = re.findall(patterns, text, flags=re.I)
                if len(found_texts) > 0:
                    pattern_list.append(1)
                else:
                    pattern_list.append(0)
            else:
                pattern_list.append(0)
        return pattern_list

    def __detect_after_adj(self, text):
        pattern_list = []
        for _, patterns in self.adj_after_keyword.items():
            if len(patterns) != 0:
                patterns = f"\s{self.irk}(?!istan|stan|ya)\s({'|'.join(patterns)})\s"
                patterns = r'{}'.format(patterns)
                found_texts = re.findall(patterns, text, flags=re.I)
                if len(found_texts) > 0:
                    pattern_list.append(1)
                else:
                    pattern_list.append(0)
            else:
                pattern_list.append(0)
        return pattern_list

    def __detect_hs_specific_verbs(self, text):
        pattern_list = []
        for _, patterns in self.hs_specific_verbs.items():
            if len(patterns) != 0:
                found_pattern = False
                patterns = f"\s{self.irk}(?!istan|stan|ya)(.*?)({'|'.join(patterns)})"
                patterns = r'{}'.format(patterns)
                found_texts = re.findall(patterns, text, flags=re.I)
                for found_text in found_texts:
                    if len(found_text[1].split()) > 13:
                        new_text = " ".join(found_text[1].split()[-13:]) + " " + found_text[2]
                        if len(re.findall(patterns, new_text, flags=re.I)) > 0:
                            found_pattern = True
                            break
                    else:
                        found_pattern = True
                        break
                if found_pattern:
                    pattern_list.append(1)
                else:
                    pattern_list.append(0)
            else:
                pattern_list.append(0)
        return pattern_list
        
    def apply_rules(self, data):
        data["special_pattern"] = data["text"].apply(lambda text: self.__detect_special_pattern(text))
        data["general_rule"] = data["text"].apply(lambda text: self.__detect_general_rules(text))
        data["anti_hs"] = data["text"].apply(lambda text: self.__detect_anti_hs(text))
        data["hs_specific_verb"] = data["text"].apply(lambda text: self.__detect_hs_specific_verbs(text))
        data["adj_bef_keyword"] = data["text"].apply(lambda text: self.__detect_bef_adj(text))
        data["adj_after_keyword"] = data["text"].apply(lambda text: self.__detect_after_adj(text))
        return data

In [252]:
rule_assigner = LinguisticRuleAssigner()

In [253]:
sample_data =  rule_assigner.apply_rules(sample_data)

[('yahudi', 'baskını'), ('yahudi', 'baskını')]
[('yahudi', 'haydutlar')]
[('ermeni', 'piçleri')]
[('suriyeli', 'vahşeti')]


In [254]:
sample_data["adj_after_pattern"] = sample_data["adj_after_keyword"].apply(lambda x: sum(x))

In [255]:
sample_data.query("adj_after_pattern > 0").sort_values("adj_after_pattern", ascending=False)

,id,date,pub_name,type,title,content,Label,sentences,text,phase,hs_specific_verb,special_pattern,general_rule,anti_hs,adj_bef_keyword,adj_after_keyword,adj_after_pattern
4764,4791,12 Ekim 2017 Perşembe,yeni akit,ulusal,mescidi aksa ya yine yahudi baskını,mescid-i aksa'ya yine yahudi baskını israil po...,hate,[mescidi aksaya yine yahudi baskını israil pol...,mescidi aksa ya yine yahudi baskını mescidi ak...,train,"[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]",0.0000,"[0, 0, 1, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 1, 0, 0]",1
7272,7305,10 Nisan 2018 Salı,milat,ulusal,ismail selvi,öteden/beriden ismail zelvi kudüs'e zeytin dal...,hate,[ötedenberiden ismail zelvi kudüse zeytin dalı...,ismail selvi ötedenberiden ismail zelvi kudüse...,train,"[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]",0.0625,"[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 1, 0, 0, 0]",1
1626,1638,06 Eylül 2016 Salı,kayseri star haber,Yerel,ahmet sıvacı,"tendürek a, dağında ağlar kurtlar kuşlar tendü...",hate,[tendürek dağında ağlar kurtlar kuşlar tendüre...,ahmet sıvacı tendürek dağında ağlar kurtlar ku...,train,"[0, 0, 0, 1, 0]","[0, 0, 0, 0, 0]",0.0000,"[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 1]",1
559,562,17 Şubat 2016 Çarşamba,adana günlük,Yerel,fa pyşg,•:vo; ** **- -. »c yine suriyeli vahşeti adana...,hate,[yine suriyeli vahşeti adanada suriyeli inşaat...,fa pyşg yine suriyeli vahşeti adanada suriyeli...,train,"[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]",0.0000,"[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 1]",1


In [252]:
sample_data.query("id == 11294").text.values

array([], dtype=object)

In [261]:
keywords = json.load(open("targeted_keywords.json", "r"))

In [260]:
"|".join([item for sublist in list(adj_bef_keyword.values()) for item in sublist])
#[item for item in degree_list for degree_list in ]

'sapıtan|çakma|facir|gizli|şımarık|kışkırmış|hırsız|katleden|korkak|yamyam|başbelası|kripto|içimizdeki|sinsi|açgözlü|dönek|ikiyüzlü|azgın|edepsiz|yağmacı|çapulcu|işgalci|gaspçı|lanetlenmiş|allahın lanetlediği|zalim|şerefsiz|gaddar|gasıp|canavarlaşmış|canavarlaşan|hain|katleden|gavur|kalleş|kan gölüne çeviren|katil|insanlık suçu işleyen|bebek katili|cani|vahşi|eli kanlı'

In [12]:
special_patterns = json.load(open("special_patterns.json", "r"))

In [21]:
pattern_list = []
for degree, patterns in special_patterns.items():
    if len(patterns) != 0:
        patterns = f"\s({'|'.join(patterns)})|({'|'.join(patterns)})\s"
        patterns = r'{}'.format(patterns)
        finded_texts = re.findall(patterns, text, flags=re.I)
        if len(finded_texts) > 0:
            pattern_list.append(1)
        else:
            pattern_list.append(0)
    else:
        pattern_list.append(0)

\s(vahşi toplumlar|batı cehaleti|haçlı zihniyeti|haçlı ruhu|kriptolar)|(vahşi toplumlar|batı cehaleti|haçlı zihniyeti|haçlı ruhu|kriptolar)\s
\s(batıl batı)|(batıl batı)\s
\s(yahudi ajanı|yahudi uşağı|kripto ermeni|suriyeli işgali|afgan işgali|mülteci işgali|pakistanlı işgali|arapların işgali)|(yahudi ajanı|yahudi uşağı|kripto ermeni|suriyeli işgali|afgan işgali|mülteci işgali|pakistanlı işgali|arapların işgali)\s
\s(yahudi çakallığı|katil rum|haydut rumlar|rum zorbalığı|kafir alevi|ateist alevi|ateist aleviler|ateist PKK)|(yahudi çakallığı|katil rum|haydut rumlar|rum zorbalığı|kafir alevi|ateist alevi|ateist aleviler|ateist PKK)\s


In [17]:
patterns

['vahşi toplumlar',
 'batı cehaleti',
 'haçlı zihniyeti',
 'haçlı ruhu',
 'kriptolar']

In [18]:
'|'.join(patterns)

'vahşi toplumlar|batı cehaleti|haçlı zihniyeti|haçlı ruhu|kriptolar'

In [141]:
re.findall(r'[rum|rus|ermeni][luk|lik].*?yap', "bugün rusluk konusunda konuşma yapan insan da çok rumluk yapamayan da çok.")

['rusluk konusunda konuşma yap', 'rumluk yap']

In [183]:
re.findall(r'(rum|fransızlar|ermeniler)(un|ın|in).*(hain|barbar)+', "bu rumun haince tutumlarından ötürü ve fransızların barbar yaklaşımları ve ermenilerin bu durmaz barbar bizi yordu.", flags=re.I|re.S)

[('rum', 'un', 'barbar')]

In [56]:
re.findall(r'(rum|fransızlar|ermeniler)(un|ın|in)(.*?)(hain|barbar)(ce|ca)?', "bu rumun haince tutumlarından ötürü ve fransızların barbar yaklaşımları ve ermenilerin bu durmaz barbar bizi yordu.")

[('rum', 'un', ' ', 'hain', 'ce'),
 ('fransızlar', 'ın', ' ', 'barbar', ''),
 ('ermeniler', 'in', ' bu durmaz ', 'barbar', '')]

In [243]:
re.findall(r"(rum|ermeni|yunan)(?!istan|stan)(cı|ci|çi|cu|cü|çü)\sterör\sörgütü", "ermenici terör örgütünde değil onlar, bildiğin yunancı terör örgütü gibi saldırıyorlar")

[('ermeni', 'ci'), ('yunan', 'cı')]

In [246]:
pattern = r"\s(rum|ermeni|yunan)(?!istan|stan)(lık|lik|luk|lük)(.*?)(yap)"
re.findall(pattern, "bu caniler yorumluk da yapar ermenilik de yapar.", flags=re.I)

[('ermeni', 'lik', ' de ', 'yap')]

In [251]:
pattern = r"\s(rum|ermeni|yunan)(?!istan|stan)(.*?)\s(rum|ermeni|yunan)(?!istan|stan)(?!istan|stan)(lığını|liğini|luğunu|lüğünü)(.*?)(yap)"
re.findall(pattern, "bu böyledir beyler rum yunanlığını yapar vazgeçmez.", flags=re.I)

[('rum', '', 'yunan', 'lığını', ' ', 'yap')]

In [71]:
data[data["text"].str.contains("a bak sen")]

,id,date,pub_name,type,title,content,Label,sentences,text,phase
731,734,07 Mart 2016 Pazartesi,ayrıntılı haber,Yerel,aslanların payları tehlikeye girince,aslanların payları tehlikeye girince yirmi yıl...,hate,[aslanların payları tehlikeye girince yirmi yı...,aslanların payları tehlikeye girince aslanları...,train
5773,5801,15 Ocak 2018 Pazartesi,batı akdeniz gazetesi,yerel,chp bildiğiniz gibi,av. ramazan yıldırım chp bildiğiniz gibi değer...,hate,[ramazan yıldırım chp bildiğiniz gibi değerli ...,chp bildiğiniz gibi ramazan yıldırım chp bildi...,train
9220,9261,11 Haziran 2019 Salı,yeni dönem,yerel,rumlara bak sen,"rumlara bak sen! kıbrıs rum yönetimi, doğu akd...",hate,[rumlara bak sen kıbrıs rum yönetimi doğu akde...,rumlara bak sen rumlara bak sen kıbrıs rum yön...,train
10567,10611,07 Kasım 2019 Perşembe,hamle niğde,yerel,bende virüs var,durmuş göktekin bende virüs var! bakıyorum hay...,hate,[durmuş göktekin bende virüs var bakıyorum hay...,bende virüs var durmuş göktekin bende virüs va...,train
11250,11294,04 Ocak 2019 Cuma,gazete ipekyol,yerel,hak ehlinin hüznü,hak ehli nin huznu ceviz at l/l t. bir arkadaş...,hate,"[hak ehli nin huznu ceviz at ll, bir arkadaşım...",hak ehlinin hüznü hak ehli nin huznu ceviz at ...,train
13588,13636,24 Mayıs 2016 Salı,türkiye,Ulusal,nerde eski tatlar,"sorduk ustasına, anlattı. nenle eski tatlar ir...",not_hate,"[sorduk ustasına anlattı, nenle eski tatlar ir...",nerde eski tatlar sorduk ustasına anlattı nenl...,train
18408,18456,15 Ocak 2018 Pazartesi,yeni asır,Ulusal,bay kemalin soykırımcı başkanı,canan kaftana oğlu pntc-an '• tarihte bugün: e...,not_hate,[canan kaftana oğlu pntcan tarihte bugün ermen...,bay kemalin soykırımcı başkanı canan kaftana o...,train


In [77]:
data.query("id == 5801").text.values

array(['chp bildiğiniz gibi ramazan yıldırım chp bildiğiniz gibi değerli okurlar demokrasi ile yönetilen birden fazla siyası partilerin bulunduğu ülkelerde demokrasinin olmazsa olmazı muhalefet partileridir bir ülkede ıyı bir muhalefet partisi aynı zamanda partileri iktidarı dizginlemek iktidarın güç zehirlenmesini önlemek açısından son derece elzemdir aklı başında bir muhalefet sadece iktidarı eleştirmez aynı zamanda yol göstericidir de on beş yıllık ak parti iktidarı döneminde aklı başında bir muhalefetin olmaması ak parti için bir şans değildir eğer aklı başında bir muhalefet partisi olsa ıdı türkiyenin geldiği nokta çok daha farklı olabilirdi ülkemizin en hayatı en stratejik konularında bile iktidar ile masaya beraberce yumruk vuran bir muhalefet partisi elbette bu ülkenin çıkarları ıçm çok çok elzemdir belki chp düzelir yapıcı bir muhalefet yapar umuduna ne zaman kapılsak hep bir hayal kırıklığı yaşadık yukarıda yazdıklarımdan varmak istediğim yer sırasıdır chp nın dün istanbul il

In [236]:
general_rules = [
    r"[IRK](lık|lik|luk|lük)(.*?)(yap)",
    r"[IRK_KIMLIK](?!istan|stan)(.*?)(skandalı)",
    r"[KIMLIK](?!istan|stan)(.*?)(işgali)",
    r"[IRK_KIMLIK](?!istan|stan)(a|e)\s(bak sen)",
    r"[IRK](?!istan|stan)(.*?)(işgali)",
    r"[IRK](?!istan|stan)(.*?)[IRK](?!istan|stan)(lığını|liğini|luğunu|lüğünü)(.*?)(yap)",
    r"[IRK](?!istan|stan)(ın|in|un|ün)(.*?)(uşağı|işbirlikçisi|piyonu|kuklası)(.*?)[IRK](?!istan|stan)",
    r"[IRK](?!istan|stan)(ın|in|un|ün)(.*?)(hain|vahşi|insanlık dışı|hunharca|kan donduran|şeytani|sinsi|teşebbüsleri|planları|oluşumları)",
    r"[IRK](?!istan|stan)(.*?)(kurşunlarıyla|bombalarıyla|parasıyla)",
    r"[IRK](?!istan|stan)(ın|in|un|ün)(.*?)(gerçekleştirdiği|yaptığı)(.*?)(katliam|zulüm|soykırım)",
    r"[IRK](?!istan|stan)\s(tarafından)\s(gerçekleştirilen|yapılan)(.*?)(saldırılar|katliam|zulüm|soykırım)",
    r"[IRK](?!istan|stan)\s(tarafından)\s[IRK](?!istan|stan)(a|e)(yönelik)?(.*?)(saldırılar|katliam|zulüm|soykırım)",
    r"[IRK](?!istan|stan)\s(tarafından)(.*?)(saldırıya|katliama|zulme|soykırıma)(.*?)(maruz kal|uğra)",
    r"[IRK]\s(destekli)(.*?)[IRK](?!istan|stan)(darbesi|saldırıları|katliamı|soykırımı)",
    r"[IRK](?!istan|stan)(.*?)(öldürdü|katletti|etnik temizlik yaptı|kirletti|bastı|şehit etti)",
    r"[IRK](?!istan|stan)\s(tarafından)(.*?)(öldürüldü|katledildi|basıldı|şehit edildi)"  
]

In [114]:
turkish_to_latin_map = {ord("ü"): "u", ord("ö"): "o", ord("ı"): "i", ord("ğ"): "g", ord("ş"): "s", ord("ç"): "c"}

In [143]:
"\s(afgan|afganlar|arap|araplar|bulgar|bulgarlar|çingene|çingeneler|çinli|çinliler|ermeni|ermeniler|ezidi|yezidiler|ezidiler|ingiliz|ingilizler|ıraklı|ıraklılar|iranlı|iranlılar|italyan|italyanlar|kırgız|kırgızlar|kurt|kurtler|kürt|kürd|kürdler|kürtler|lübnanlı|lübnanlılar|macar|macarlar|musevi|museviler|pakistanlı|paki|pakiler|pakistanlılar|rum|rumlar|rus|moskof|ruslar|sırp|sırplı|sırplılar|sırplar|suriyeli|suri|suriler|suriyeliler|süryani|süryaniler|türkmen|türkmenler|yahudi|yahudiler|yemenli|yemenliler|yezid|yezit|yezitler|yezidler|yunan|yunanlı|yunanlılar|yunanlar)(?!istan|stan|ya)(ın|in|un|ün)\s(\w*)\s?(uşağı|işbirlikçisi|piyonu|kuklası)(\\w*)\\s(afgan|afganlar|arap|araplar|bulgar|bulgarlar|çingene|çingeneler|çinli|çinliler|ermeni|ermeniler|ezidi|yezidiler|ezidiler|ingiliz|ingilizler|ıraklı|ıraklılar|iranlı|iranlılar|italyan|italyanlar|kırgız|kırgızlar|kurt|kurtler|kürt|kürd|kürdler|kürtler|lübnanlı|lübnanlılar|macar|macarlar|musevi|museviler|pakistanlı|paki|pakiler|pakistanlılar|rum|rumlar|rus|moskof|ruslar|sırp|sırplı|sırplılar|sırplar|suriyeli|suri|suriler|suriyeliler|süryani|süryaniler|türkmen|türkmenler|yahudi|yahudiler|yemenli|yemenliler|yezid|yezit|yezitler|yezidler|yunan|yunanlı|yunanlılar|yunanlar)(?!istan|stan|ya)\s".translate(turkish_to_latin_map)

'\\s(afgan|afganlar|arap|araplar|bulgar|bulgarlar|cingene|cingeneler|cinli|cinliler|ermeni|ermeniler|ezidi|yezidiler|ezidiler|ingiliz|ingilizler|irakli|iraklilar|iranli|iranlilar|italyan|italyanlar|kirgiz|kirgizlar|kurt|kurtler|kurt|kurd|kurdler|kurtler|lubnanli|lubnanlilar|macar|macarlar|musevi|museviler|pakistanli|paki|pakiler|pakistanlilar|rum|rumlar|rus|moskof|ruslar|sirp|sirpli|sirplilar|sirplar|suriyeli|suri|suriler|suriyeliler|suryani|suryaniler|turkmen|turkmenler|yahudi|yahudiler|yemenli|yemenliler|yezid|yezit|yezitler|yezidler|yunan|yunanli|yunanlilar|yunanlar)(?!istan|stan|ya)(in|in|un|un)\\s(\\w*)\\s?(usagi|isbirlikcisi|piyonu|kuklasi)(\\w*)\\s(afgan|afganlar|arap|araplar|bulgar|bulgarlar|cingene|cingeneler|cinli|cinliler|ermeni|ermeniler|ezidi|yezidiler|ezidiler|ingiliz|ingilizler|irakli|iraklilar|iranli|iranlilar|italyan|italyanlar|kirgiz|kirgizlar|kurt|kurtler|kurt|kurd|kurdler|kurtler|lubnanli|lubnanlilar|macar|macarlar|musevi|museviler|pakistanli|paki|pakiler|pakistanli

In [168]:
sample_text = "ve atatürksuriyelileradalet mi selam ve saygılar sevgili bayram ve isparta okullar ve atatürksuriyelileradalet mi selam ve saygılar sevgili gönül dostlarım kurban bayramı sebebi ile geçtiğimiz hafta yazıma ara verdim geçmiş kurban bayramınız kutlu olsun sağlıkla nice bayramları sevdiklerinizle yaşamayı mevlam nasip etsin tabii ki eski bayramlar daha farklı kutlanırdı bu sene tatil güne çıkınca bayram olarak değil tatil olarak geçti bir çoğumuz bayram ziyareti yerine soluğu sahilde aldık ne de olsa sırtımızda ki beynimizde ki maddi ve manevi yük bizleri oldukça yordu gelecekte eski bayramlarla ilgili yazılarım olur ne yazık ki bu bayram da trafik canavarı iş başında idi yaklaşık vatandaşı mız trafik kazalarında can verdi giden canların yanı sıra yaralılar ve maddi kayıplar yol denetimlerinde bayram tatili yapamayan emniyet ve asker görevlilere teşekkürler ederim her ne kadar iktidar trafik anarşisinin azaldığı için mutlu oluyorsa da aslında hiç olmamalı idi trafik kazalarında yorgunluk uykusuzluk dikkatsizlik yolların bozuk olması araçlarda bakım eksikliği rol oynuyorsa da esas olan kural tanımamak ve aceleci olmak bu arada bayram öncesi biteceği söylenen migros kavşağı alt üst geçit inşaatı yılan hikayesine döndü halen de devam ediyor isparta da akp liler iktidarlarındaki yatırımları anlatamamaktan dertliler işte örnek ortada karayollarının yaptığı çevre yolu senede bitti ama hala yama ile tamir ediliyor müze bakımına daha başlanmadı müze ziyareti tatilde demiryolları bakımı bitmedi ankara izmir istanbul tren seferleri hala rüya antalya ve konya yollarının duble yolları halen inşaat durumunda yarım veya başlanılmayan bir çok yatırım sahi toki nin emekli ve engelliler için yapacağı evler ne alemde üzerinden seçim ve oylama geçti oy sezonunda gündeme geliyor sonra proje rafa isparta cezalı sanki isparta nın milletvekilinden si akp li bakanlık bile bize düşmedi isparta belediyemiz yazdır atatürk parkı nı tadilatla geçiriyor inşallah baharında ve sonrası faaliyete geçer de bizler çay içme fırsatı buluruz sevgili gönül dostlarım haftaya okullar açılacak artık aileleri kışa hazırlık ve okul telaşları içindeler milli eğitim bakanlığı geçmişte and ımızı ve derslerde de vatan bayrak cumhuriyet küpe takan saç uzatan bir genç otobüse biner yaşlı teyze kızım kenara geçer misin der cenç iteyzecim ben kız değilim der ve anda teyze bombayı patlatır vah yavrum bu yaşta dul mu kaldın şehit gazi konularını kısıtlamıştı bu senede iktidarda ki akp nin milli eğitim bakanı atatürk atatürk ilke ve inkilapları atatürkçülük konularını sınıfa kadar müfredattan çıkardı beden eğitimi dersinde atatürkçülük işlenecekmiş aslında her ders te atatürk vardır lütfen bu konuda duyarlı olalım cumhuriyetimiz kurucusu kurtuluş savaşı komutanı mustafa kemal atatürk çocuklarımıza biz anlatalım milli değerleri evlatlarımıza kazandırmak artık ailelere bir görev imam hatip liselerine karşı değilim bu vesile ile de meslek liselerini savunuyorum imam hatip lisesi ticaret meslek lisesi kız meslek lisesi endüstri meslek lisesi eğitimin temel taşlarıdır ayırımcılık yaparak proje okulu seçilen imam hatip lisesi öğrencilerine verilecek aylık tl lik burs diğer okullarda da pilot okul statüsü ile verilemez mi bu arada suriye lilere vatandaşlık verilmeye başladı atanamayan birçok öğretmen gençlerimiz var iken suriyeli öğretmenleri atamak vicdanlara uygun mu yavaş yavaş tıp başta olmak üzere diğer mesleklerde de atamalar yolda vatandaşlık için sıkıntı evli ve fazla çocukları olması imiş bu sebeble belki ülkemizde çok evlilikte yasallaşır kuma kavgaları zaman kalkar sevgili gönül dostları da temmuz ihtilal denemesini yaşadık ölenler ve yaralananlar oldu tabii ki suçlu olanlar cezasını çekecek ama kurunun yanında yaş yakmayalım kurban bayramı öncesi meslekten fıkra zamanı minik kız elinde karnesiyle eve gelmiş karnesini babasına göstermiş babası bir bakmış baştan aşağı pekiyi bir iki tane de iyi var yalnız öğretmen karnenin altına şöyle bir not düşmüş çok akıllı ve yetenekli bir çocuk fakat bir kusuru var derste çok konuşuyor buna nasıl son verebileceğimiz hakkında fikirlerim var en kısa zamanda siz velisiyle de paylaşmak istiyorum baba bunun üzerine karneyi imzalamış ve öğretmenin görüşlerinin altına bir not düşmüş lütfen paylaşalım çünkü işe yararsa ben de annesine uygulayacağım aşk olsun ihraç edilen sağlıkçı bir bayanın intiharı yürekleri burktu mevlam günahlarını afetsin her yerde olan bu fetö cüler niye siyasette hele akp nin içinde yok bilinmezliği çözemedim hapishaneler dolu olunca çeşitli sebeblerle af uygulanmaya başladı tl lik taahüt ihlali kapalı cezaevinde ay hapis yatmayı gerektiriyor başka suçtan olan ise yıl affa uğruyor adalet bu galiba kurban bayramı öncesi vatan şaşmaz cinayeti türkiyenin gündemine oturdu sanki başka cinayetler yoktu yasak aşk cinayeti basına sakız oldu ne yazık ki ülkemde gündeme taşınacak konular bazen basın tarafından ört bas ediliyor yunan işgalinde ki ada gündem de mi kuzey ırak ta referandum niye cinayet kadar konuşul muyor tartışıl mıyor iktidar da ki akp nin ab ve abd ile restleşmesi niye basında bolca yer almıyor sevgili gönül dostlarım bir sonraki yazımda buluşmak üzere şen ve esen kalınız saygılarımla yiğitçe koray başyiğit tvvitter korayhoca instagram mustafa koray başyiğit ol atssn güneş orda re bozulmuş düşttgnlar yel gibi yaşa hlllafua kemal paşa yaaadtyrılacak yaşa mustafa amnuıl pnşn ymjuyııjln yaşılaemk mflpnrflf izmirin dağlanattn ataraum kaldım şehit olanları defin yazdım öksüz yavrularıpağrıma bastım kemal paşa yaşmadın yazılacak mücevher brmtgmal paşa yaşaadın yazılacak müavheflm".translate(turkish_to_latin_map)
sample_text2 = "yahudiler mescidi aksa ya girdi yahudiler mescidi aksaya girdi yüzlerce fanatik yahudi girmeleri yasak olan mescidi aksanın avlusunu işgal etti kudüste son tahrik fanatik yahudiler mescidi aksaya girdi israil bölgede gerginliğin artması için her yolu deniyor yahudi yerleşimci girmeleri yasak olan mescidi aksanın avlusunu işgal etti olaya tepki gösteren filistinli çocuk gözaltına alındı milletvekili baskına onculuk etti israilli aşın sağcı milletvekili yehuda glick de yerleşimciler ile birlikte mescidi aksaya girdi baskında ise filistinlilerin arandığı ve kimlik kartlarına el konulduğu iddia ediidi israilin yüdır ablukada tuttuğu gazze şeridinde filistinlinin öldüğü kişinin ise yaralandığı saldırıların ardından dün kudüste yeni bir gerginlik yaşandı kadar yahudi yerleşimci mescidi aksanın bulunduğu alanda birinci ve ikinci tapınakın yıkımını anmak amacıyla bölgeyi işgal etti yahudi yerleşimciler heykelin yıkımı adlı bayramlarının münasebetiyle girmeleri yasak olan mescidi aksaya israil ordusu ve polis eşliğinde girdi fas kapısından caminin avlusuna giren yerleşimcilere filistinli bir genç bayrak açarak tepki gösterdi bu sırada israil ordusu filistinli genci gözaltına aldı yahudi yerleşimcilerin hamas üe israil arasında dün sağlanan kırılgan ateşkes sonrası müslümanların kutsal alanına girmesi ise provokasyon olarak nitelendirildi mescidi aksanın kontrolünü elinde bulunduran ürdün de yahudi yerleşimcilerin haksiz işgalini kınadı sınır açılabilir öte yandan israil savunma bakam avigdor liebermann düzenlenen gösteriler nedeniyle temmuzdan bu yana kapalı tutulan gazze sınır kapılarının yeniden açılabileceği sinyalini verdi liebermann pazar ve pazartesi günleri sakin geçtiği takdirde kerem şalom sınır kapısı salı günü normal faaliyetlerine yeniden başlayacak diye konuştu".translate(turkish_to_latin_map)
sample_text3 = "yunan cumhurbaşkanı ege de işgal ettikleri adamıza çıktı adamızı işgal ettiler üstüne bir de şov yaptılar yunan cumhurbaşkanı egede işgal ettikleri adamıza aktı iktidar sustukça yunan iyice azıyor bodrumun karşısındaki türkiyeye ait keçi adasına giden cumhurbaşkanı yunan askerlerini denetiedi adaya yunan bayrağı çektiler daha önce adamızın işgaline sessiz kalan ankaradan çıt yok yunanistan cumhurbaşkanı prokopis pavlopulos gün önce helikopterle kalimnos adasına gitti buradan askeri botla türkiyenin keçi adasına geçti türkiye ege denizinde adeta hezimet yaşıyor yunanistan adamızı akpnin yıllık iktidarında sessizce işgal etti buralara asker ve silah yığdı bayrağım dikti türkiye ise sadece seyretti ancak yunan durmuyor ada muğla ya bağlı ama yunanistan cumhurbaşkanı ve başkomutanı pavlopulos geçen salı türkiyeye ait olan ancak işgal altındaki keçi adasına gitti yunan bayrağının dalgalandığı adada askerlerine moral ve plaket verdi saygı öztürkün haberi da bodrum işgal ettikleri adamıza bir de kilise yaptılar yunan cumhurbaşkanı türkiyeye karşı askeri yığınak yapı lan bölgeleri gezdi onun için kilise önünde tören yapıldı ligi ıp ış mi ınliılı daha önce adamızın işgaline sessiz kalan ankaradan çıt yok işgal ettikleri adamıza bir de kilise yaptılar yunanistan cumhurbaşkanı prokopis pavlopulos gün önce helikopterle yunan cumhurbaşkanı türkiyeye karşı askeri yığınak yapı kalimnos adasına gitti buradan askeri botla türkiyenin keçi adasına geçti an bölgeleri gezdi onun için kilise önünde tören yapıldı adasjbşbistanköy yunan cumhurbaşkanı işgal altındaki adamızı ziyaret etti egede tahrik zirvede kendi sularımızdaki adanın yunanistan tarafından işgal edildiğ bir kez daha kanıtlandı cumhurbaşkanı pavlopulos başkomutan sıfatıyla muğlaya bağlı keçi adasına gitti saye oztjrk bodrum asadece mıluzakuctaii emekli kurmay albay ümit yalım muğla keçi adasımn millik türk karasuları içinde olduğunu söyledi yalım ada hakkında şu bilgileri verdi bodruma olan mesafesi mil keçi adası istanbuldaki büyükadanm iki misli büyüklükte adanın her tarafında yunan bayrağı dalgalanıyor adaya yunan askerleri ile birlikte top ve uçaksavai gibi ağır silahlar yerleştirilmiş keçi adası silah ve mühimmat deposu haline getirilerek üeri karakol haline dönüştürülmüş ardak krizinin ardından ege denizindeki adanın yunanistan tarafından işgal edilmesinin yankılan sürerken bir skandal daha yaşandı yunanistan cumhurbaşkanı prokopis pavlopulosun geçen salı günü muğlaya bağlı keçi adasım ziyaret ettiği ortaya çıktl yunan silahlı kuvvetleri tarafından işgal edilen adamızdan birisi olan keçi adasında yunanistan bayrağı dalgalanırken türk yetkilileri bu konuda da sessiz kaldı sözde vahye plaket bile verdi ziyaretin görüntüleri yunanistan basınında yer aldı görüntülerin başında prokopis pavlopulos bakanlar ve askeri yetkililer önce kalimnos adasında görülüyor yunan heyeti daha sonra askeri botla keçi adasına geçiyor cumhurbaşkanı ve başkomutan sıfatıyla adaya ayak basan pavlopulos askeri tahkimat yapılan kale bölgesine de gitti genelkurmay başkam üe birlikte işgalci yunan askerlerini denetledi pavlopulos adada yaptırılan kilisenin önünde adalar valisi tuğgeneral sergentaniye plaket verdi bu durum adalarımız ilhak edildiği gibi yunanistanın atadığı askeri valilerin yönetimine terk edilmiş yorumlarına neden oldu pavlopulos kendi basınına keçi adasımn yunan sınırlan içinde olduğunu iddia etti yunan cumhurbaşkanı adada türkiyenin lozan antlaşmasına yunanistanın sınırlarına ve toprak bütünlüğüne saygı duymasını istedi tsknın görevi engelleniyor milli savunma bakanlığı eski genel sekreteri emekli kurmay albay ümit yalım skandalla ilgili konuştu yalım lozan antlaşmasına göre başta keçi adası olmak üzere yunan işgali altında olan toplam türk adası ve türk kayalığının türkiyenin egemenliği altında olduğunu söyledi yunan cumhurbaşkanı bakanlan ve komuta kademesinin elini kolunu sallayıp keçi adasına gittiğini belirten yalım tskmn görevini yapmasının engellendiği anlaşılıyor böylece türk adası ve türk kayalığının yunan askerine alenen teslim edildiği bir kez daha tescillenmiştir diye konuştu yunan cumhurbaşkanı işgal altındaki adamızı ziyaret etti egede tahrik zirvede kendi sularımızdaki adanın yunanistan tarafından işgal edildiğ bir kez daha kanıtlandı cumhurbaşkanı pavlopulos başkomutan sıfatıyla muğlaya bağlı keçi adasına gitti vyy tçardak krizinin ardından ege bjpvs ıtop nml ffintfntm nmi ili ti eş denizindeki adanın yu ij edilmesinin yankılan sürerken bir skandal daha yaşandı yuna ftfl ir nistan cumhurbaşkanı prokopis il pavlopulosun geçen salı günü muğlaya jft bağlı keçi adasım ziyaret ettiği ortaya çık jbi ii mbıı tı yunan silahlı kuvvetleri tarafından işgal jdüfc edilen adamızdan birisi olan keçi adasın jha da yunanistan bayrağı dalgalanırken türk yetkilileri bu konuda da sessiz kaldl ziyaretin görüntüleri yunanistan basınınyunan cumhurbaşkanı da yer aldı görüntülerin başında prokopis prokopis pavlopulos pavlopulos bakanlar ve askeri yetkililer önce paskeritörenl karşılandı kalimnos adasında görülüyor yunan heyeti daha sonra askeri botla keçi adasına geçiyor cumhurbaşkam ve başkomutan sıfatıyla adaya ayak basan pavlopulos askeri tahkimat bodrumasadece ljıj lkj nnm yapılan kale bölgesine de gitti genelkurmay tia ııı başkam üe birlikte işgalci yunan askerlerini mil uzakllinalfeii denetledi pavlopulos adada yaptırılan kilise emekli kurmay albay ümit yalım önünde adalar valisi tuğgeneral sergenta muğla keçi adasımn mitlik türk ka niye plaket verdi bu durum adalarımız ilhak rasulan içinde olduğunu söyledi yalım diği gibi yunanistanm atadığı askeri valile ada hakkında şu bilgileri verdi bod rin yönetimine terk edilmiş yorumlarına ne ruma olan mesafesi mil keçi adası den oldu pavlopulos kendi basınına keçi ada istanbuldaki büyükadanm iki misli sırrnı yunan sınırlan içinde olduğunu iddia büyüklükte adanın her tarafında yunan etti yunan cumhurbaşkam adada türkiyenin bayrağı dalgalanıyor adaya yunan as lozan antlaşmasına yunanistanın sınırlarına kerleri ile birlikte top ve uçaksavar gibi ve toprak bütünlüğüne saygı duymasını istedi ağır silahlar yerleştirilmiş keçi adası si lah ve mühimmat deposu haline getirile hlifspıhimi inıııiiuaaaauıiiım rek üeri karakol haline dönüştürülmüş müli savunma bakanlığı eski genel sekreteri".translate(turkish_to_latin_map)
sample_rule = r"(yahudiler|yahudi|yunan)\s(.*?)\s?(isgali)".translate(turkish_to_latin_map)
re.findall(sample_rule, sample_text3, flags=re.I)

[('yunan',
  'cumhurbaskani ege de isgal ettikleri adamiza cikti adamizi isgal ettiler ustune bir de sov yaptilar yunan cumhurbaskani egede isgal ettikleri adamiza akti iktidar sustukca yunan iyice aziyor bodrumun karsisindaki turkiyeye ait keci adasina giden cumhurbaskani yunan askerlerini denetiedi adaya yunan bayragi cektiler daha once adamizin',
  'isgali'),
 ('yunan',
  'durmuyor ada mugla ya bagli ama yunanistan cumhurbaskani ve baskomutani pavlopulos gecen sali turkiyeye ait olan ancak isgal altindaki keci adasina gitti yunan bayraginin dalgalandigi adada askerlerine moral ve plaket verdi saygi ozturkun haberi da bodrum isgal ettikleri adamiza bir de kilise yaptilar yunan cumhurbaskani turkiyeye karsi askeri yiginak yapi lan bolgeleri gezdi onun icin kilise onunde toren yapildi ligi ip is mi inliili daha once adamizin',
  'isgali'),
 ('yunan',
  'cumhurbaskani turkiyeye karsi askeri yiginak yapi kalimnos adasina gitti buradan askeri botla turkiyenin keci adasina gecti an bolgele

In [162]:
sample_text3

'yunan cumhurbaskani ege de isgal ettikleri adamiza cikti adamizi isgal ettiler ustune bir de sov yaptilar yunan cumhurbaskani egede isgal ettikleri adamiza akti iktidar sustukca yunan iyice aziyor bodrumun karsisindaki turkiyeye ait keci adasina giden cumhurbaskani yunan askerlerini denetiedi adaya yunan bayragi cektiler daha once adamizin isgaline sessiz kalan ankaradan cit yok yunanistan cumhurbaskani prokopis pavlopulos gun once helikopterle kalimnos adasina gitti buradan askeri botla turkiyenin keci adasina gecti turkiye ege denizinde adeta hezimet yasiyor yunanistan adamizi akpnin yillik iktidarinda sessizce isgal etti buralara asker ve silah yigdi bayragim dikti turkiye ise sadece seyretti ancak yunan durmuyor ada mugla ya bagli ama yunanistan cumhurbaskani ve baskomutani pavlopulos gecen sali turkiyeye ait olan ancak isgal altindaki keci adasina gitti yunan bayraginin dalgalandigi adada askerlerine moral ve plaket verdi saygi ozturkun haberi da bodrum isgal ettikleri adamiza bi

In [149]:
sample_rule

'\\s(yahudi|yahudiler|yunan)(?!istan|stan|ya)\\s(\\w*)\\s?(isgali)'